In [25]:
# Завдання 1
# Ще раз розберіть усі приклади до уроку, повторіть теорію та ознайомтеся з документацією щодо розглянутих модулів.
# Завдання 2
# Створіть список цілих чисел. Отримайте список квадратів непарних чисел із цього списку.
arr = [3, 4, 7, 9, -1, 2]
res = filter(lambda x: x % 2 != 0, arr)
res = map(lambda x: x**2, res)
print(list(res))

[9, 49, 81, 1]


In [26]:
# Завдання 3
# Створіть функцію-генератор чисел Фібоначчі. Застосуйте до неї декоратор, який залишатиме в послідовності лише парні числа.
def even_filter(func):
    def wrapper(*args, **kwargs):
        for number in func(*args, **kwargs):
            if number % 2 == 0:
                yield number
    return wrapper

@even_filter
def fibonacci(n):
    a, b = 0, 1
    for i in range(n):
        yield a
        a, b = b, a + b
        
fib = fibonacci(10)
print(list(fib))

[0, 2, 8, 34]


In [2]:
# Завдання 3
# Створіть функцію-генератор чисел Фібоначчі. Застосуйте до неї декоратор, який залишатиме в послідовності лише парні числа.
def even_filter(func):
    def wrapper(*args, **kwargs):
        number = func(*args, **kwargs)
        if number % 2 == 0:
            yield number
    return wrapper

@even_filter
def fibonacci(n):
    a, b = 0, 1
    for i in range(n):
        yield a
        a, b = b, a + b
        
fib = fibonacci(10)
print(list(fib))

TypeError: unsupported operand type(s) for %: 'generator' and 'int'

In [27]:
# Завдання 4
# Створіть звичайну функцію множення двох чисел. 
# Створіть каррувану функцію множення двох чисел. Частково застосуйте її до одного аргументу, до двох аргументiв
mult = lambda x, y: x*y
mult_carr = lambda x: lambda y: x * y
print(mult(2, 5))
print(mult_carr(2)(5))
mult3 = mult_carr(3)
for i in range(5):
    print(mult3(i), end = ' ')

10
10
0 3 6 9 12 

In [30]:
# Напишіть декоратор, який буде для переданої функції заміряти час виконання.
# Напишіть програму яка буде виводити 25 перших чисел Фібоначі, використовуючи
# для цього три наведені в тексті заняття функції - без кеша, з кешем довільної
# довжини, з кешем з модулю functools з максимальною кількістю 10 елементів.
# За допомогою написаного Вами декоратора заміряйте і порівняйте швидкість 
# роботи цих трьох варіантів.

import functools
import time
from typing import Callable, Any


def debug(func):
    """Друкує аргументи функції, і результат іі виконання. Повертає результат роботи функції."""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]
        signature = ", ".join(args_repr + kwargs_repr)
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")
        return value
    return wrapper

def timer(func: Callable) -> Any:
    """Print the runtime of the decorated function."""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        runtime = end_time - start_time
        print(f"Done! {func.__name__} in {runtime} secs")
        return value

    return wrapper_timer



@timer
def long_loop(num_times):
    for _ in range(num_times):
        sum([i**3 for i in range(1000000)])
        
long_loop(5)

Done! long_loop in 0.7471767000024556 secs


In [33]:
 import functools

def cache(func):
    """Keep a cache of previous function calls"""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        cache_key = args + tuple(kwargs.items())
        if cache_key not in wrapper.cache:
            wrapper.cache[cache_key] = func(*args, **kwargs)
        return wrapper.cache[cache_key]
    wrapper.cache = dict()
    return wrapper

def fibonacci_simple(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)


@cache
def fibonacci_cached(num):
    return fibonacci_simple(num)

@functools.lru_cache(maxsize=10)
def fibonacci(num):
    return fibonacci_simple(num)

@debug
@timer
def test_fibonacci(max_num, func):
    for i in range(max_num + 1):
        print(func(i), end = ' ')
    print()

test_fibonacci(25, fibonacci_simple)
test_fibonacci(25, fibonacci_cached)
test_fibonacci(25, fibonacci)

Calling test_fibonacci(25, <function fibonacci_simple at 0x0000026F49AD7D80>)
0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 
Done! test_fibonacci in 0.00010409999958937988 secs
'test_fibonacci' returned None
Calling test_fibonacci(25, <function fibonacci_cached at 0x0000026F495A2520>)
0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 
Done! test_fibonacci in 0.00013179999950807542 secs
'test_fibonacci' returned None
Calling test_fibonacci(25, <functools._lru_cache_wrapper object at 0x0000026F49BAC510>)
0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 
Done! test_fibonacci in 8.24999988253694e-05 secs
'test_fibonacci' returned None
